## Preprocess SL simulations

Function of script
* Extract sl simulations for point locations & reformat.

Context in workflow
* Needs an updated station list (metadata.csv with sl_mm_per_yr) but otherwise not dependent on the water level data preparation steps.

In [ ]:
import os
import pandas as pd
import xarray as xr
from pathlib import Path

scens = ['ssp126', 'ssp245', 'ssp370', 'ssp585']

DATA = Path("data")
INPUTS = DATA / "inputs"
OUTPUTS = DATA / "outputs"

stnlist_csv_edit = OUTPUTS / "metadata.csv"
input_dir = INPUTS / "sl"
output_dir = OUTPUTS / "sl_proc"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
stnlist = pd.read_csv(stnlist_csv_edit, encoding='latin1')

for stn_num in stnlist['stn_num'].values:
    print(stn_num)

    sl_mm_per_yr = stnlist[stnlist['stn_num'] == stn_num]['sl_mm_per_yr'].values[0]
    lat = stnlist[stnlist['stn_num'] == stn_num]['lat'].values[0]
    lon = stnlist[stnlist['stn_num'] == stn_num]['lon'].values[0]
    
    out_file = os.path.join(output_dir, f'{stn_num:05d}_sl.nc')
    
    scen_list = []
    for scen  in scens:
        ncfile_in = os.path.join(input_dir, f'AR6withGeodeticVertical_medium_{scen}.nc')
        all_sl = xr.open_dataset(ncfile_in)
        sl_this_station = all_sl.sel(lat=lat, lon=lon, method='nearest')
        scen_list.append(sl_this_station)
    all_scens = xr.concat(scen_list, dim = 'SSP')

    sl = all_scens.rename({"Cumulative_Probability": "quantile", "SSP":"experiment_id", "Sea_Levels" : "sl_delta"})
    sl.sl_delta.attrs['sl_mm_yr'] = sl_mm_per_yr
    sl.sl_delta.attrs['ref_period'] = [1995, 2014]
    sl.sl_delta.attrs.update(description=sl.attrs.get('description', ''))
    ssps = sl.experiment_id.str.split(dim="match", sep="_").isel(match=1)
    sl = sl.assign_coords(experiment_id=ssps)
    sl = sl.drop_vars('spatial_ref')

    sl.to_netcdf(out_file)